In [ ]:
from datasets import load_dataset,Dataset,concatenate_datasets 
from glob import glob
import os,torch,random
from shutil import copyfile
cache_dir = '' 
model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"

os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [ ]:
template_dataset = load_dataset("wnut_17",cache_dir=cache_dir)
label_list = ['O','B-phenotype','I-phenotype','B-coding_system','I-coding_system','B-code','I-code']

In [ ]:
def data_loader(files):
    data_dict = {'id':[],'tokens':[],'ner_tags':[]}
    i=0
    for file in files:
        with open(file,'r') as f:
            lines = f.read().splitlines()
            tokens = []
            tags = []
            for line in lines:
                if line != '':
                    token,tag = line.split('\t')
                    tokens.append(token)
                    if tag!='O':
                        tag = tag.split('-')[0]+'-'+tag.split('-')[1].lower()
                    tags.append(label_list.index(tag))
                else:
                    data_dict['id'].append(str(i))
                    data_dict['tokens'].append(tokens)
                    data_dict['ner_tags'].append(tags)
                    tokens = []
                    tags = []
                    i+=1
    return data_dict

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
import numpy as np
import evaluate

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {}
for i,label in enumerate(label_list):
    id2label.update({i:label})
label2id = {}
for i,label in enumerate(label_list):
    label2id.update({label:i})

### human annotation only

In [ ]:
train_files = glob(f'./data/NER/human_annotated/train.bio')
valid_files = glob(f'./data/NER/human_annotated/dev.bio')
test_files = glob(f'./data/NER/human_annotated/test.bio')
train = Dataset.from_dict(data_loader(train_files))
valid = Dataset.from_dict(data_loader(valid_files))
test = Dataset.from_dict(data_loader(test_files))

template_dataset['train'] = train
template_dataset['validation'] = valid
template_dataset['test'] = test
        
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
tokenized_dataset = template_dataset.map(tokenize_and_align_labels, batched=True)
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id,cache_dir=cache_dir
)
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir=f"./models/NER_BiomedBERT/",
    learning_rate=5e-6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    do_train = True,
    do_eval = True,
    do_predict = True,
    metric_for_best_model = 'f1'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
trainer.save_model()

predictions, labels, metrics = trainer.predict(tokenized_dataset['test'], metric_key_prefix="predict")

predictions = np.argmax(predictions, axis=2)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

In [ ]:
from ner_metrics import classification_report
from pprint import pprint
y_true = []
y_pred = []
for sentence, predictions,golds in zip(template_dataset['test']['tokens'], true_predictions, template_dataset['test']['ner_tags']):
    for token,tag,gold in zip(sentence,predictions,golds):
        y_true.append(label_list[gold])
        y_pred.append(tag)
lenient = classification_report(tags_true=y_true, tags_pred=y_pred, mode="lenient") # for lenient match
strict = classification_report(tags_true=y_true, tags_pred=y_pred, mode="strict")
print ('entity','\t','precision','\t','recall','\t','f1-score')
for entity in strict.keys():
    scores = strict[entity]
    print (entity,'\t',scores['precision'],'\t',scores['recall'],'\t',scores['f1-score'])
for entity in lenient.keys():
    scores = lenient[entity]
    print (entity,'\t',scores['precision'],'\t',scores['recall'],'\t',scores['f1-score'])

### synthetic

In [ ]:
train_files = glob(f'./data/NER/synthetic/train.bio')
valid_files = glob(f'./data/NER/synthetic/dev.bio')
test_files = glob(f'./data/NER/human_annotated/test.bio')
train = Dataset.from_dict(data_loader(train_files))
valid = Dataset.from_dict(data_loader(valid_files))
test = Dataset.from_dict(data_loader(test_files))

template_dataset['train'] = train
template_dataset['validation'] = valid
template_dataset['test'] = test
        
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
tokenized_dataset = template_dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
tokenized_dataset = template_dataset.map(tokenize_and_align_labels, batched=True)
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id,cache_dir=cache_dir
)
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir=f"./models/NER_BiomedBERT_synthetic/",
    learning_rate=5e-6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    do_train = True,
    do_eval = True,
    do_predict = True,
    metric_for_best_model = 'f1'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
trainer.save_model()

predictions, labels, metrics = trainer.predict(tokenized_dataset['test'], metric_key_prefix="predict")

predictions = np.argmax(predictions, axis=2)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

with open(f'./results/NER_BiomedBERT_synthetic_all.txt','w') as output_file:
    for sentence, predictions,golds in zip(template_dataset['test']['tokens'], true_predictions, template_dataset['test']['ner_tags']):
        for token,tag,gold in zip(sentence,predictions,golds):
            output_file.write(f'{token}\t{label_list[gold]}\t{tag}\n')
        output_file.write('\n')         
        
!python /data/yhu5/CLAMP/melaxdev-deepmeddocker_tf-28b7a60e460b/dockersimple/evaluate_jianfu_new.py -lf ./results/NER_BiomedBERT_synthetic_all.txt

from ner_metrics import classification_report
from pprint import pprint
y_true = []
y_pred = []
for sentence, predictions,golds in zip(template_dataset['test']['tokens'], true_predictions, template_dataset['test']['ner_tags']):
    for token,tag,gold in zip(sentence,predictions,golds):
        y_true.append(label_list[gold])
        y_pred.append(tag)
lenient = classification_report(tags_true=y_true, tags_pred=y_pred, mode="lenient") # for lenient match
strict = classification_report(tags_true=y_true, tags_pred=y_pred, mode="strict")
print ('entity','\t','precision','\t','recall','\t','f1-score')
for entity in strict.keys():
    scores = strict[entity]
    print (entity,'\t',scores['precision'],'\t',scores['recall'],'\t',scores['f1-score'])
for entity in lenient.keys():
    scores = lenient[entity]
    print (entity,'\t',scores['precision'],'\t',scores['recall'],'\t',scores['f1-score'])

In [ ]:
train_files = glob(f'./data/NER/human_annotated/train.bio')
valid_files = glob(f'./data/NER/human_annotated/dev.bio')
test_files = glob(f'./data/NER/human_annotated/test.bio')
train = Dataset.from_dict(data_loader(train_files))
valid = Dataset.from_dict(data_loader(valid_files))
test = Dataset.from_dict(data_loader(test_files))

template_dataset['train'] = train
template_dataset['validation'] = valid
template_dataset['test'] = test
        
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
tokenized_dataset = template_dataset.map(tokenize_and_align_labels, batched=True)
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForTokenClassification.from_pretrained(
    f"./models/NER_BiomedBERT_synthetic/", num_labels=len(label_list), id2label=id2label, label2id=label2id,cache_dir=cache_dir
)
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir=f"./models/NER_BiomedBERT_synthetic_continual/",
    learning_rate=5e-6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    do_train = True,
    do_eval = True,
    do_predict = True,
    metric_for_best_model = 'f1'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
trainer.save_model()

predictions, labels, metrics = trainer.predict(tokenized_dataset['test'], metric_key_prefix="predict")

predictions = np.argmax(predictions, axis=2)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]


from ner_metrics import classification_report
from pprint import pprint
y_true = []
y_pred = []
for sentence, predictions,golds in zip(template_dataset['test']['tokens'], true_predictions, template_dataset['test']['ner_tags']):
    for token,tag,gold in zip(sentence,predictions,golds):
        y_true.append(label_list[gold])
        y_pred.append(tag)
lenient = classification_report(tags_true=y_true, tags_pred=y_pred, mode="lenient") # for lenient match
strict = classification_report(tags_true=y_true, tags_pred=y_pred, mode="strict")
print ('entity','\t','precision','\t','recall','\t','f1-score')
for entity in strict.keys():
    scores = strict[entity]
    print (entity,'\t',scores['precision'],'\t',scores['recall'],'\t',scores['f1-score'])
for entity in lenient.keys():
    scores = lenient[entity]
    print (entity,'\t',scores['precision'],'\t',scores['recall'],'\t',scores['f1-score'])